Imports

In [81]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
from scipy.stats import mode
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

warnings.filterwarnings('ignore')

## Données
Charger les données

In [82]:
# Chargement du fichier
def chargement_fichier(f):
    data = pd.read_csv("data/" + f, sep=r"\s+", engine="python", header=None)

    # Séparation des variables
    X = data.iloc[:, :-1].astype(float)  # récupère les  colonnes de données
    y = data.iloc[:, -1]                 # dernière colonne (cible)

    print("X shape :", X.shape)
    print("y shape :", y.shape)
    return X, y


Découpage des données en apprentissage et test

In [83]:
def decoupage_donnees(X, y):
    # Découpage stratifié
    X_A, X_T, y_A, y_T = train_test_split(
        X, y,
        test_size=1/3,
        stratify=y,
        random_state=42
    )

    print("Taille apprentissage :", X_A.shape)
    print("Taille test :", X_T.shape)
    return X_A, X_T, y_A, y_T


Affichage des metrics de résultats de test

In [84]:
def display_metrics(y_T_enc, y_pred_mlp_norm):    
    accuracy_mlp = accuracy_score(y_T_enc, y_pred_mlp_norm)
    print("Accuracy MLP :", accuracy_mlp)

    conf_matrix_mlp = confusion_matrix(y_T_enc, y_pred_mlp_norm)
    print("Matrice de confusion MLP :\n", conf_matrix_mlp)

    # Précision pour chaque classe
    precision = precision_score(y_T_enc, y_pred_mlp_norm, average=None, zero_division=0)
    print("\nPrécision par classe:")
    for i, p in enumerate(precision):
        print(f"  Classe {i+1}: {p:.4f}")

    # Rappel pour chaque classe
    recall = recall_score(y_T_enc, y_pred_mlp_norm, average=None, zero_division=0)
    print("\nRappel par classe:")
    for i, r in enumerate(recall):
        print(f"  Classe {i+1}: {r:.4f}")

### PERCEPTRON MULTI CLASSE
Sans normalisation

In [85]:
def evaluation_perceptron_multiclasse(X_A, X_T, y_A, y_T):

    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    perceptron = Perceptron(
        max_iter=1000,
        eta0=0.1,
        random_state=42
    )

    perceptron.fit(X_A, y_A_enc)

    y_pred = perceptron.predict(X_T)

    print("=== PERCEPTRON MULTICLASSE SANS NORMALISATION ===")

    display_metrics(y_T_enc, y_pred)




Avec normalisation

In [86]:
def evaluation_perceptron_multiclasse_normalise(X_A, X_T, y_A, y_T):

    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)
    scaler = StandardScaler()

    # Apprentissage de la normalisation sur A
    X_A_norm = scaler.fit_transform(X_A)

    # Application de la même normalisation sur T
    X_T_norm = scaler.transform(X_T)

    perceptron = Perceptron(
        max_iter=1000,
        eta0=0.1,
        random_state=42
    )

    perceptron.fit(X_A_norm, y_A_enc)

    y_pred_perceptron_norm = perceptron.predict(X_T_norm)

    print("=== PERCEPTRON MULTICLASSE AVEC NORMALISATION ===")

    display_metrics(y_T_enc, y_pred_perceptron_norm)




### PERCEPTRON MULTI COUCHE
Sans normalisation

In [87]:
def evaluation_mlp(X_A, X_T, y_A, y_T):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    mlp = MLPClassifier(
        hidden_layer_sizes=(3,),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=10
    )

    mlp.fit(X_A, y_A_enc)
    y_pred_mlp = mlp.predict(X_T)

    print("=== MLP SANS NORMALISATION ===")

    display_metrics(y_T_enc, y_pred_mlp)

Avec normalisation

In [88]:
def evaluation_mlp_normalise(X_A, X_T, y_A, y_T):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)
    scaler = StandardScaler()

    # Apprentissage de la normalisation sur A
    X_A_norm = scaler.fit_transform(X_A)

    # Application de la même normalisation sur T
    X_T_norm = scaler.transform(X_T)

    mlp_norm = MLPClassifier(
        hidden_layer_sizes=(3,),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=42
    )

    mlp_norm.fit(X_A_norm, y_A_enc)

    y_pred_mlp_norm = mlp_norm.predict(X_T_norm)

    print("=== MLP AVEC NORMALISATION ===")

    display_metrics(y_T_enc, y_pred_mlp_norm)


### Evaluation des performances

In [ ]:
def traintement(f):
    X, y = chargement_fichier(f)
    X_A, X_T, y_A, y_T = decoupage_donnees(X, y)
    evaluation_perceptron_multiclasse(X_A, X_T, y_A, y_T)
    evaluation_perceptron_multiclasse_normalise(X_A, X_T, y_A, y_T)
    evaluation_mlp(X_A, X_T, y_A, y_T)
    evaluation_mlp_normalise(X_A, X_T, y_A, y_T)

traintement("Wave.txt")

X shape : (5000, 40)
y shape : (5000,)
Taille apprentissage : (3333, 40)
Taille test : (1667, 40)
=== PERCEPTRON MULTICLASSE SANS NORMALISATION ===
Accuracy MLP : 0.8554289142171566
Matrice de confusion MLP :
 [[451  50  63]
 [ 29 514   8]
 [ 32  59 461]]

Précision par classe:
  Classe 1: 0.8809
  Classe 2: 0.8250
  Classe 3: 0.8665

Rappel par classe:
  Classe 1: 0.7996
  Classe 2: 0.9328
  Classe 3: 0.8351
=== PERCEPTRON MULTICLASSE AVEC NORMALISATION ===
Accuracy MLP : 0.8044391121775645
Matrice de confusion MLP :
 [[448  77  39]
 [ 38 444  69]
 [ 83  20 449]]

Précision par classe:
  Classe 1: 0.7873
  Classe 2: 0.8207
  Classe 3: 0.8061

Rappel par classe:
  Classe 1: 0.7943
  Classe 2: 0.8058
  Classe 3: 0.8134
=== MLP SANS NORMALISATION ===
Accuracy MLP : 0.8650269946010798
Matrice de confusion MLP :
 [[476  48  40]
 [ 34 488  29]
 [ 47  27 478]]

Précision par classe:
  Classe 1: 0.8546
  Classe 2: 0.8668
  Classe 3: 0.8739

Rappel par classe:
  Classe 1: 0.8440
  Classe 2: 0.

### BAGGING
Echantillonage des données d'apprentissage

In [90]:
def bootstrap_sample(X, y):
    n = X.shape[0]
    indices = np.random.choice(n, size=n, replace=True)
    return X[indices], y[indices]


Apprentissage avec Bagging avec K MLP

In [91]:
def train_bagging_mlp(X, y, K):
    models = []

    for k in range(K):
        X_boot, y_boot = bootstrap_sample(X, y)

        mlp = MLPClassifier(
            hidden_layer_sizes=(3,),
            activation='relu',
            solver='adam',
            max_iter=1000,
            random_state=k
        )

        mlp.fit(X_boot, y_boot)
        models.append(mlp)

    return models


Prédiction avec Bagging

In [92]:
def bagging_predict(models, X):
    # Prédictions de tous les modèles
    all_preds = np.array([model.predict(X) for model in models])

    # Vote majoritaire
    y_pred, _ = mode(all_preds, axis=0)
    return y_pred.ravel()


Evaluation avec Bagging

In [93]:
def evaluation_bagging_mlp(X_A, X_T, y_A, y_T, K):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    X_A = np.asarray(X_A)
    X_T = np.asarray(X_T)
    y_A_enc = np.asarray(y_A_enc)
    y_T_enc = np.asarray(y_T_enc)

    # Entraînement du bagging
    models = train_bagging_mlp(X_A, y_A_enc, K)

    # Prédictions agrégées
    y_pred_bagging = bagging_predict(models, X_T)

    # Évaluation
    print("=== BAGGING DE MLP ===")

    display_metrics(y_T_enc, y_pred_bagging)


Code de traitement pour Bagging

In [94]:
def traintement_bagging(f, K=10):
    X, y = chargement_fichier(f)
    X_A, X_T, y_A, y_T = decoupage_donnees(X, y)
    evaluation_bagging_mlp(X_A, X_T, y_A, y_T, K)

traintement_bagging("iris.txt", K=10)

X shape : (150, 4)
y shape : (150,)
Taille apprentissage : (100, 4)
Taille test : (50, 4)
=== BAGGING DE MLP ===
Accuracy MLP : 0.96
Matrice de confusion MLP :
 [[16  0  0]
 [ 0 15  2]
 [ 0  0 17]]

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 1.0000
  Classe 3: 0.8947

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 0.8824
  Classe 3: 1.0000
